In [1]:
import pandas as pd
import numpy as np
import os
import torch
from torch.utils.data import Dataset
import nibabel as nib
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import glob
from torchvision import transforms

In [2]:
df = pd.read_excel('all_patients.xlsx')
df = df[df.shape_VoxelVolume > 125]
df.drop(df[df['labels'] == 3].index, inplace = True) ## dropping label of the call OTHERS
df.drop(df[df['number_annotations'] > 1].index, inplace = True)
df

,subjid,label,shape_Elongation,shape_Flatness,shape_LeastAxisLength,shape_MajorAxisLength,shape_Maximum2DDiameterColumn,shape_Maximum2DDiameterRow,shape_Maximum2DDiameterSlice,shape_Maximum3DDiameter,...,ngtdm_Busyness,ngtdm_Coarseness,ngtdm_Complexity,ngtdm_Contrast,ngtdm_Strength,Age,number_annotations,F,M,labels
0,Oslo01,1,0.829356,0.682981,8.512717,12.464063,14.866069,14.560220,12.529964,15.394804,...,158.977235,0.008929,0.302273,0.074659,0.008950,58,1,0,1,0
1,Oslo01,2,0.944143,0.900681,14.133927,15.692487,17.804494,19.416488,18.681542,19.467922,...,39.445794,0.007407,0.115448,0.010426,0.007130,58,1,0,1,0
4,Oslo02,4,0.849295,0.841082,8.703783,10.348315,10.816654,11.704700,11.704700,12.449900,...,14.239428,0.022868,0.159307,0.020316,0.023465,50,1,1,0,1
5,Oslo02,5,0.939828,0.821847,12.093804,14.715396,17.262677,17.888544,17.492856,18.000000,...,49.609354,0.008536,0.137901,0.028455,0.008212,50,1,1,0,1
7,Oslo03,1,0.930698,0.629604,10.914986,17.336268,20.615528,18.439089,19.416488,20.712315,...,43.648474,0.007781,0.123220,0.017415,0.007982,64,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,Stan_328,1,0.810571,0.325076,8.769420,26.976501,31.384710,28.160256,27.459060,33.075671,...,12.988427,0.019891,0.030104,0.000639,0.019768,53,1,0,1,1
2415,Stan_328,8,0.967496,0.597282,6.131668,10.265944,11.661904,12.369317,12.165525,12.688578,...,18.452502,0.024791,0.208465,0.046975,0.023205,53,1,0,1,1
2418,Stan_328,11,0.561746,0.506390,5.004740,9.883181,8.246211,11.704700,10.000000,12.124356,...,0.553323,0.468447,0.025263,0.000748,0.365681,53,1,0,1,1
2419,Stan_328,12,0.959775,0.756903,5.164755,6.823538,7.615773,8.602325,8.602325,8.602325,...,0.000000,1000000.000000,0.000000,0.000000,0.000000,53,1,0,1,1


In [3]:
new_df = pd.DataFrame({'Subjid': df.iloc[:, 0], 'Tumor': df.iloc[:, 1], 'Labels': df.iloc[:, -1]})
new_df = new_df.drop_duplicates(subset='Subjid', keep='first')
#new_df.to_excel('labels_DL.xlsx', index = False)
new_df

,Subjid,Tumor,Labels
0,Oslo01,1,0
4,Oslo02,4,1
7,Oslo03,1,0
11,Oslo04,1,0
13,Oslo06,1,1
...,...,...,...
2359,Stan_318,1,1
2369,Stan_319,2,1
2387,Stan_323,3,1
2408,Stan_328,1,1


In [4]:
stan = new_df['Subjid'][59:].to_list()
oslo = new_df['Subjid'][:59].to_list()

## we have created a new folder called deep learning with all the folder with all the folder per patient labels as 0, 1 or 2 with tumors greater than 125mm3

In [324]:
# Specify the path to the parent folder
parent_folder = '/data/projects/TMOR/data/Deeplearning/'

# Get a list of all subfolders within the parent folder
subfolders = [f.path for f in os.scandir(parent_folder) if f.is_dir()]

# Iterate over the subfolders and rename them

for folder in subfolders:
    folder_name = os.path.basename(folder)
    new_folder_name = folder_name.replace("Subject", "Oslo")
    #new_folder_name = folder_name.replace("Mets", "Stan")
    new_folder_path = os.path.join(parent_folder, new_folder_name)
    os.rename(folder, new_folder_path)


In [325]:
#delete the other file for OSLO
directory = '/data/projects/TMOR/data/Deeplearning/'

# Get a list of all patient folders
patient_folders = glob.glob(os.path.join(directory, 'Oslo*'))

for patient_folder in patient_folders:
    #id = patient_folder[-2:]
    seg_folders = glob.glob(os.path.join(patient_folder, 'seg*'))
    t1_gds = glob.glob(os.path.join(patient_folder, 't1_gd*'))
    #print(seg_folders)
    # Iterate over each seg folder
    for seg_folder in seg_folders:
        for t1_gd in t1_gds:
            #print(seg_folder)
            image_files = glob.glob(os.path.join(patient_folder, '*'))
            #print(image_files)
            # Delete all the image files except for 'seg_1'
            for file in image_files:
            # Check if the file is in the segmentation folder or T1-Gd folder
                if file not in [seg_folder, t1_gd]:
                    os.remove(file)

In [326]:
patient_folders = glob.glob(os.path.join(directory, 'Oslo*'))
import shutil
for patient_folder in patient_folders:
    id = patient_folder[-6:]
    if id not in oslo:
        shutil.rmtree(patient_folder)

In [196]:
import os

directory = '/data/projects/TMOR/data/Deeplearning/'

patient_folders = glob.glob(os.path.join(directory, 'Oslo*'))


# Iterate over patient folders
for patient_folder in patient_folders:
    #patient_path = os.path.join(directory, patient_folder)
    seg_folders = glob.glob(os.path.join(patient_folder, 'seg*'))
    # Rename segmentation files
    for seg_folder in seg_folders:
        seg_old_path = seg_folder
        seg_new_path = os.path.join(patient_folder, 'seg.nii.gz')
        os.rename(seg_old_path, seg_new_path)
        #print(seg_new_path)

## STANDFORD DATA

In [328]:
# Specify the path to the parent folder
parent_folder = '/data/projects/TMOR/data/Deeplearning/'

# Get a list of all subfolders within the parent folder
subfolders = [f.path for f in os.scandir(parent_folder) if f.is_dir()]

# Iterate over the subfolders and rename them

for folder in subfolders:
    folder_name = os.path.basename(folder)
    new_folder_name = folder_name.replace("Mets", "Stan")
    new_folder_path = os.path.join(parent_folder, new_folder_name)
    os.rename(folder, new_folder_path)


In [329]:
import os
import glob
#delete the other file for OSLO
directory = '/data/projects/TMOR/data/Deeplearning/'

# Get a list of all patient folders
patient_folders = glob.glob(os.path.join(directory, 'Stan*'))

for patient_folder in patient_folders:
    seg_folders = glob.glob(os.path.join(patient_folder, 'seg*'))
    t1_gds = glob.glob(os.path.join(patient_folder, 't1_gd*'))
    # Iterate over each seg folder
    for seg_folder in seg_folders:
        for t1_gd in t1_gds:
            image_files = glob.glob(os.path.join(patient_folder, '*'))
            # Delete all the image files except for 'seg_1'
            for file in image_files:
                # Check if the file is in the segmentation folder or T1-Gd folder
                if file not in [seg_folder, t1_gd]:
                    os.remove(file)


In [330]:
#Remove all the files that are not within stan
import shutil
for patient_folder in patient_folders:
    id = patient_folder[-8:]
    if id not in stan:
        shutil.rmtree(patient_folder)

## move all t1_pre to the right folder of /data/projects/TMOR/data/Deeplearning/A/

## OSLO

In [28]:
# Specify the path to the parent folder
parent_folder = '/data/projects/TMOR/data/OsloPreprocessed/OsloPreprocessed/'

# Get a list of all subfolders within the parent folder
subfolders = [f.path for f in os.scandir(parent_folder) if f.is_dir()]

# Iterate over the subfolders and rename them

for folder in subfolders:
    folder_name = os.path.basename(folder)
    new_folder_name = folder_name.replace("Subject", "Oslo")
    #new_folder_name = folder_name.replace("Mets", "Stan")
    new_folder_path = os.path.join(parent_folder, new_folder_name)
    os.rename(folder, new_folder_path)

In [44]:
import shutil
directory = '/data/projects/TMOR/data/OsloPreprocessed/OsloPreprocessed/'

patient_folders = glob.glob(os.path.join(directory, 'Oslo*'))

repository = '/data/projects/TMOR/data/Deeplearning/'

pacientes = glob.glob(os.path.join(repository, 'Oslo*'))

for patient_folder in patient_folders:
    t1_pre_files = glob.glob(os.path.join(patient_folder, 't1_pre*'))
    for t1 in t1_pre_files:
        patient_id = os.path.basename(patient_folder)
        patient_match = next((p for p in pacientes if patient_id in p), None)
        if patient_match:
        # Move each t1-pre file to the respective patient folder
            for t1_pre_file in t1_pre_files:
                destination_folder = os.path.join(patient_match, os.path.basename(t1_pre_file))
                shutil.copy(t1_pre_file, destination_folder)

## Standford

In [45]:
# Specify the path to the parent folder
parent_folder = '/data/projects/TMOR/data/StanfordPreprocessed/StanfordPreprocessed/'

# Get a list of all subfolders within the parent folder
subfolders = [f.path for f in os.scandir(parent_folder) if f.is_dir()]

# Iterate over the subfolders and rename them

for folder in subfolders:
    folder_name = os.path.basename(folder)
    #new_folder_name = folder_name.replace("Subject", "Oslo")
    new_folder_name = folder_name.replace("Mets", "Stan")
    new_folder_path = os.path.join(parent_folder, new_folder_name)
    os.rename(folder, new_folder_path)

In [46]:
directory = '/data/projects/TMOR/data/StanfordPreprocessed/StanfordPreprocessed/'

patient_folders = glob.glob(os.path.join(directory, 'Stan*'))

repository = '/data/projects/TMOR/data/Deeplearning/'

pacientes = glob.glob(os.path.join(repository, 'Stan*'))

for patient_folder in patient_folders:
    t1_pre_files = glob.glob(os.path.join(patient_folder, 't1_pre*'))
    for t1 in t1_pre_files:
        patient_id = os.path.basename(patient_folder)
        patient_match = next((p for p in pacientes if patient_id in p), None)
        if patient_match:
        # Move each t1-pre file to the respective patient folder
            for t1_pre_file in t1_pre_files:
                destination_folder = os.path.join(patient_match, os.path.basename(t1_pre_file))
                shutil.copy(t1_pre_file, destination_folder)

## class object

In [34]:
def crop_to_largest_tumor(tensor, crop_size):
    # Get tumor dimensions
    tumor_size = np.sum(tensor > 0, axis=(1, 2, 3))
    max_tumor_size = np.max(tumor_size)

    # Calculate cropping indices
    start = (tumor_size - max_tumor_size) // 2
    end = start + max_tumor_size

    # Perform cropping
    cropped_tensor = tensor[:, start:end, start:end, start:end]

    return cropped_tensor
class BrainSegmentationDataset(Dataset):
    def __init__(self, img_dir, labels_file, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.img_labels = pd.read_excel(labels_file)
        self.transform = transform
        self.target_transform = target_transform

        self.X_train, self.X_test, self.y_train, self.y_test = self.split_data()

    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, idx):
        # Load the mask directly
        seg_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 'seg.nii.gz')
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 't1_gd.nii.gz')
        data_1 = nib.load(img_path)
        data = data_1.get_fdata()
        mask_1 = nib.load(seg_path)
        mask = mask_1.get_fdata()
        label = self.y_train[idx]

        data = torch.from_numpy(data).unsqueeze(0)
        mask = torch.from_numpy(mask).unsqueeze(0)
        label = torch.tensor(label).long()

        return mask, data, label

    def split_data(self):
        X = self.img_labels.iloc[:, 0].values
        y = self.img_labels.iloc[:, 2].values

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

        X_train = self.process_images(X_train)
        X_test = self.process_images(X_test)

        return X_train, X_test, y_train, y_test

    def process_images(self, image_names):
        image_tensors = []

        for image_name in image_names:
            # Load the mask directly
            seg_path = os.path.join(self.img_dir, image_name, 'seg.nii.gz')
            img_path = os.path.join(self.img_dir, image_name, 't1_gd.nii.gz')
            data_1 = nib.load(img_path)
            data = data_1.get_fdata()
            mask_1 = nib.load(seg_path)
            mask = mask_1.get_fdata()

            data_tensor = torch.from_numpy(data).unsqueeze(0)
            mask_tensor = torch.from_numpy(mask).unsqueeze(0)

            image_tensors.append((data_tensor, mask_tensor))

        return image_tensors

            tumor_size = np.sum(mask > 0, axis=(0, 1, 2))
            print(tumor_size)
            max_tumor_size = np.max(tumor_size)
            print(max_tumor_size)

            # Calculate cropping indices
            start = (tumor_size - max_tumor_size) // 2
            print(start)
            end = start + max_tumor_size

            # Crop image and mask tensors
            data = data[start[0]:end[0], start[1]:end[1], start[2]:end[2]]
            mask = mask[start[0]:end[0], start[1]:end[1], start[2]:end[2]]

            data_tensor = torch.from_numpy(data).unsqueeze(0)
            mask_tensor = torch.from_numpy(mask).unsqueeze(0)

            image_tensors.append((data_tensor, mask_tensor))

        return image_tensors

img_dir = '/data/projects/TMOR/data/Deeplearning/'
labels_file = 'labels_DL.xlsx'
transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])

dataset = BrainSegmentationDataset(img_dir, labels_file, transform=transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)


1344
1344
0


IndexError: invalid index to scalar variable.

## lets divide the tumors

In [104]:
seg_dir = '/data/projects/TMOR/data/Deeplearning/'
#indices = []
for subj_id in os.listdir(seg_dir):
    #extrasct the correct id from the subjects
    # Define paths to the segmentation/mask and regular t1 postprocessed files for this subject
    seg_glob = os.path.join(seg_dir, f'{subj_id}', f'seg.nii.gz')
    reg_glob = os.path.join(seg_dir, f'{subj_id}', f't1_gd.nii.gz')
    seg_paths = glob.glob(seg_glob)
    reg_paths = glob.glob(reg_glob)
    # Loop through all matching segmentation and registration files
    for seg, reg in zip(seg_paths, reg_paths):
        if os.path.exists(seg) and os.path.exists(reg):
            print(seg)
            print(reg)
            mask = nib.load(seg)
            vol_mask = mask.get_fdata()
            for label in np.unique(vol_mask):
                if label != 0:
                    label_indices = np.argwhere(vol_mask == label)
                    if len(label_indices) > 125:
                        indices = label_indices
                        X_min, X_max = np.min(indices[:, 0]), np.max(indices[:, 0])
                        Y_min, Y_max = np.min(indices[:, 1]), np.max(indices[:, 1])
                        Z_min, Z_max = np.min(indices[:, 2]), np.max(indices[:, 2])
                        seg_partitioned = vol_mask[X_min:X_max, Y_min:Y_max, Z_min:Z_max]

                        output_dir = os.path.join(seg_dir, 'final', f'{subj_id}')
                        os.makedirs(output_dir, exist_ok=True)

                        # Save the seg_partitioned into a file
                        output_path = os.path.join(output_dir, 'seg_seg_' + f'{str(label)}' +'.nii.gz')
                        nib.save(nib.Nifti1Image(seg_partitioned, mask.affine), output_path)

#for label_indices in indices:
        # Do something with the label indices
        #print(f"Label: {label_indices[0, 3]}, Length: {len(label_indices)}")

/data/projects/TMOR/data/Deeplearning/Oslo02/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo02/t1_gd.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo03/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo03/t1_gd.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo04/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo04/t1_gd.nii.gz
/data/projects/TMOR/data/Deeplearning/Stan_230/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Stan_230/t1_gd.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo06/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo06/t1_gd.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo07/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo07/t1_gd.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo08/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo08/t1_gd.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo09/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo09/t1_gd.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo10/seg.nii.gz
/data/projects/TMOR/data/Dee

KeyboardInterrupt: 

In [105]:
D_x_max, D_y_max, D_z_max = 0, 0, 0
seg_dir = '/data/projects/TMOR/data/Deeplearning/'
#indices = []
for subj_id in os.listdir(seg_dir):
    #extrasct the correct id from the subjects
    # Define paths to the segmentation and regular t1 postprocessed files for this subject
    seg_glob = os.path.join(seg_dir, f'{subj_id}', f'seg.nii.gz')
    reg_glob = os.path.join(seg_dir, f'{subj_id}', f't1_gd.nii.gz')
    seg_paths = glob.glob(seg_glob)
    reg_paths = glob.glob(reg_glob)
    # Loop through all matching segmentation and registration files
    for seg, reg in zip(seg_paths, reg_paths):
        if os.path.exists(seg) and os.path.exists(reg):
            mask = nib.load(seg)
            vol_mask = mask.get_fdata()
            for label in np.unique(vol_mask):
                if label != 0:
                    label_indices = np.argwhere(vol_mask == label)
                    if len(label_indices) > 125:
                        indices = label_indices
                        X_min, X_max = np.min(indices[:, 0]), np.max(indices[:, 0])
                        Y_min, Y_max = np.min(indices[:, 1]), np.max(indices[:, 1])
                        Z_min, Z_max = np.min(indices[:, 2]), np.max(indices[:, 2])
                        #we need the differences now boi
                        D_x = X_max - X_min
                        D_y = Y_max - Y_min
                        D_z = Z_max - Z_min
                        if D_x_max < D_x:
                        	D_x_max = D_x
                        if D_y_max < D_y:
                        	D_y_max = D_y
                        if D_z_max < D_z:
                        	D_z_max = D_z

In [107]:
drop_dir = '/data/projects/TMOR/data/'
seg_dir = '/data/projects/TMOR/data/Deeplearning/'
#indices = []
for subj_id in os.listdir(seg_dir):
    #extrasct the correct id from the subjects
    # Define paths to the segmentation and regular t1 postprocessed files for this subject
    seg_glob = os.path.join(seg_dir, f'{subj_id}', f'seg.nii.gz')
    reg_glob = os.path.join(seg_dir, f'{subj_id}', f't1_gd.nii.gz')
    reg2_glob = os.path.join(seg_dir, f'{subj_id}', f't1_pre.nii.gz')
    seg_paths = glob.glob(seg_glob)
    reg_paths = glob.glob(reg_glob)
    reg2_paths = glob.glob(reg2_glob)
    # Loop through all matching segmentation and registration files
    for seg, reg, reg2 in zip(seg_paths, reg_paths, reg2_paths):
        if os.path.exists(seg) and os.path.exists(reg) and os.path.exists(reg2):
            mask = nib.load(seg)
            vol_mask = mask.get_fdata()
            t1_gd = nib.load(reg)
            vol_t1_gd = t1_gd.get_fdata()
            t1_pre = nib.load(reg2)
            vol_t1_pre = t1_pre.get_fdata()
            for label in np.unique(vol_mask):
                if label != 0:
                    label_indices = np.argwhere(vol_mask == label)
                    if len(label_indices) > 125:
                        indices = label_indices
                        X_min, X_max = np.min(indices[:, 0]), np.max(indices[:, 0])
                        Y_min, Y_max = np.min(indices[:, 1]), np.max(indices[:, 1])
                        Z_min, Z_max = np.min(indices[:, 2]), np.max(indices[:, 2])

                        X_avg = (X_min + X_max) // 2
                        Y_avg = (Y_min + Y_max) // 2
                        Z_avg = (Z_min + Z_max) // 2
                        # Pad the indices with zeros
                        X_low = max(X_avg - (D_x_max // 2), 0)
                        X_high = min(X_avg + (D_x_max // 2), vol_mask.shape[0])
                        Y_low = max(Y_avg - (D_y_max // 2), 0)
                        Y_high = min(Y_avg + (D_y_max // 2), vol_mask.shape[1])
                        Z_low = max(Z_avg - (D_z_max // 2), 0)
                        Z_high = min(Z_avg + (D_z_max // 2), vol_mask.shape[2])

                        pene = vol_mask[X_low:X_high, Y_low:Y_high, Z_low:Z_high]
                        t1_desp = vol_t1_gd[X_low:X_high, Y_low:Y_high, Z_low:Z_high]
                        t1_antes = vol_t1_pre[X_low:X_high, Y_low:Y_high, Z_low:Z_high]

                        ## define the pads
                        pad_width = [
                            (0, max(D_x_max - pene.shape[0], 0)),
                            (0, max(D_y_max - pene.shape[1], 0)),
                            (0, max(D_z_max - pene.shape[2], 0))
                        ]
                        # define the partitions
                        seg_partitioned = np.pad(pene, pad_width, mode='constant', constant_values=0)
                        t1_gd_partitioned = np.pad(t1_desp, pad_width, mode='constant', constant_values=0)
                        t1_pre_partitioned = np.pad(t1_antes, pad_width, mode='constant', constant_values=0)
                        #
                        output_dir = os.path.join(drop_dir, 'VeryFinal', subj_id, str(int(label)))
                        os.makedirs(output_dir, exist_ok=True)
                        # tumors segemented of the segmentation of mask
                        output_path_1 = os.path.join(output_dir, 'seg.nii.gz')
                        nib.save(nib.Nifti1Image(seg_partitioned, mask.affine), output_path_1)
                        # tumors segmented of the t1_gd
                        output_path_2 = os.path.join(output_dir, 't1_gd.nii.gz')
                        nib.save(nib.Nifti1Image(t1_gd_partitioned, t1_gd.affine), output_path_2)
                        # tumors segemented if t1_pre
                        output_path_3 = os.path.join(output_dir, 't1_pre.nii.gz')
                        nib.save(nib.Nifti1Image(t1_pre_partitioned, t1_pre.affine), output_path_3)

In [125]:
seg = f'/data/projects/TMOR/data_v2/Oslo/Subject66/seg_66.nii.gz'

mask = nib.load(seg)
vol_mask = mask.get_fdata()
np.unique(vol_mask)
for label in np.unique(vol_mask):
    if label != 0:
        label_indices = np.argwhere(vol_mask == label)
        if len(label_indices) > 125:
            print(label)

0.9999999997671694


In [129]:
df2.loc['Oslo66']

label                     1.000000
shape_Elongation          0.860610
shape_Flatness            0.856119
shape_LeastAxisLength     7.661783
shape_MajorAxisLength     8.949440
                           ...    
Age                      78.000000
number_annotations        1.000000
F                         1.000000
M                         0.000000
labels                    0.000000
Name: Oslo66, Length: 113, dtype: float64

In [127]:
seg = f'/data/projects/TMOR/data_v2/Oslo/Subject65/seg_65.nii.gz'

mask = nib.load(seg)
vol_mask = mask.get_fdata()
np.unique(vol_mask)
for label in np.unique(vol_mask):
    if label != 0:
        label_indices = np.argwhere(vol_mask == label)
        if len(label_indices) > 125:
            print(label)

1.0
2.0
3.0


In [128]:
df2.loc['Oslo65']

,label,shape_Elongation,shape_Flatness,shape_LeastAxisLength,shape_MajorAxisLength,shape_Maximum2DDiameterColumn,shape_Maximum2DDiameterRow,shape_Maximum2DDiameterSlice,shape_Maximum3DDiameter,shape_MeshVolume,...,ngtdm_Busyness,ngtdm_Coarseness,ngtdm_Complexity,ngtdm_Contrast,ngtdm_Strength,Age,number_annotations,F,M,labels
subjid,,,,,,,,,,,,,,,,,,,,,
Oslo65,1,0.855166,0.800732,20.767148,25.935220,28.301943,26.683328,29.832868,31.048349,6924.333333,...,204.346342,0.001758,0.163391,0.027139,0.001712,65,1,1,0,1
Oslo65,2,0.842210,0.762170,6.820878,8.949284,9.486833,10.000000,10.295630,10.677078,317.291667,...,1.579231,0.164355,0.037328,0.000915,0.160495,65,1,1,0,1
Oslo65,3,0.973216,0.814385,21.189772,26.019365,34.365681,34.000000,37.589892,38.716921,8639.791667,...,255.052021,0.001400,0.164962,0.026721,0.001384,65,1,1,0,1


In [116]:
df2.loc['Stan_098']

,label,shape_Elongation,shape_Flatness,shape_LeastAxisLength,shape_MajorAxisLength,shape_Maximum2DDiameterColumn,shape_Maximum2DDiameterRow,shape_Maximum2DDiameterSlice,shape_Maximum3DDiameter,shape_MeshVolume,...,ngtdm_Busyness,ngtdm_Coarseness,ngtdm_Complexity,ngtdm_Contrast,ngtdm_Strength,Age,number_annotations,F,M,labels
subjid,,,,,,,,,,,,,,,,,,,,,
Stan_098,1,0.738644,0.686814,12.638130,18.401099,20.880613,20.880613,17.088007,21.908902,2199.291667,...,30.766956,0.009691,0.093001,0.008930,0.009700,61,1,1,0,1
Stan_098,3,0.951212,0.816792,5.247428,6.424437,7.280110,7.615773,7.810250,8.124038,120.375000,...,0.309877,0.835931,0.018547,0.000678,0.519532,61,1,1,0,1


In [52]:
import numpy as np

seg_dir = '/data/projects/TMOR/data/Deeplearning/'

D_x_max, D_y_max, D_z_max = 0, 0, 0

for subj_id in os.listdir(seg_dir):
    seg_glob = os.path.join(seg_dir, f'{subj_id}', 'seg.nii.gz')
    reg_glob = os.path.join(seg_dir, f'{subj_id}', 't1_gd.nii.gz')
    seg_paths = glob.glob(seg_glob)
    reg_paths = glob.glob(reg_glob)

    for seg, reg in zip(seg_paths, reg_paths):
        if os.path.exists(seg) and os.path.exists(reg):
            mask = nib.load(seg)
            vol_mask = mask.get_fdata()

            for label in np.unique(vol_mask):
                if label != 0:
                    label_indices = np.argwhere(vol_mask == label)
                    if len(label_indices) > 125:
                        indices = label_indices
                        X_min, X_max = np.min(indices[:, 0]), np.max(indices[:, 0])
                        Y_min, Y_max = np.min(indices[:, 1]), np.max(indices[:, 1])
                        Z_min, Z_max = np.min(indices[:, 2]), np.max(indices[:, 2])

                        X_avg = (X_min + X_max) // 2
                        Y_avg = (Y_min + Y_max) // 2
                        Z_avg = (Z_min + Z_max) // 2

                        D_x = X_max - X_min
                        D_y = Y_max - Y_min
                        D_z = Z_max - Z_min

                        if D_x_max < D_x:
                            D_x_max = D_x
                        if D_y_max < D_y:
                            D_y_max = D_y
                        if D_z_max < D_z:
                            D_z_max = D_z

                        X_low = max(X_avg - (D_x_max // 2), 0)
                        X_high = min(X_avg + (D_x_max // 2), vol_mask.shape[0])
                        Y_low = max(Y_avg - (D_y_max // 2), 0)
                        Y_high = min(Y_avg + (D_y_max // 2), vol_mask.shape[1])
                        Z_low = max(Z_avg - (D_z_max // 2), 0)
                        Z_high = min(Z_avg + (D_z_max // 2), vol_mask.shape[2])

                        pene = vol_mask[X_low:X_high, Y_low:Y_high, Z_low:Z_high]

                        pad_width = [
                            (0, max(D_x_max - pene.shape[0], 0)),
                            (0, max(D_y_max - pene.shape[1], 0)),
                            (0, max(D_z_max - pene.shape[2], 0))
                        ]

                        seg_partitioned = np.pad(pene, pad_width, mode='constant', constant_values=0)

                        if seg_partitioned.shape != pene.shape:
                            print('SUBJECT', f'{subj_id}', 'WITH', f'{label}', 'WAS PADDED')

                        output_dir = os.path.join(seg_dir, 'C', subj_id)
                        os.makedirs(output_dir, exist_ok=True)

                        output_path = os.path.join(output_dir, f'{label}_seg.nii.gz')
                        nib.save(nib.Nifti1Image(seg_partitioned, mask.affine), output_path)

SUBJECT Oslo02 WITH 5.0 WAS PADDED
SUBJECT Oslo03 WITH 1.0 WAS PADDED
SUBJECT Oslo03 WITH 3.0 WAS PADDED
SUBJECT Oslo04 WITH 1.0 WAS PADDED
SUBJECT Stan_230 WITH 1.0 WAS PADDED
SUBJECT Oslo06 WITH 1.0 WAS PADDED
SUBJECT Oslo07 WITH 1.0 WAS PADDED
SUBJECT Oslo08 WITH 3.0 WAS PADDED
SUBJECT Oslo09 WITH 1.0 WAS PADDED
SUBJECT Oslo09 WITH 2.0 WAS PADDED
SUBJECT Oslo10 WITH 1.0 WAS PADDED
SUBJECT Oslo11 WITH 1.0 WAS PADDED
SUBJECT Oslo12 WITH 2.0 WAS PADDED
SUBJECT Oslo13 WITH 1.0 WAS PADDED
SUBJECT Oslo14 WITH 1.0 WAS PADDED
SUBJECT Oslo17 WITH 1.0 WAS PADDED
SUBJECT Oslo18 WITH 1.0 WAS PADDED
SUBJECT Oslo18 WITH 2.0 WAS PADDED
SUBJECT Oslo18 WITH 3.0 WAS PADDED
SUBJECT Oslo19 WITH 1.0 WAS PADDED
SUBJECT Oslo20 WITH 1.0 WAS PADDED
SUBJECT Oslo20 WITH 2.0 WAS PADDED
SUBJECT Oslo21 WITH 1.0 WAS PADDED
SUBJECT Oslo22 WITH 1.0 WAS PADDED
SUBJECT Oslo23 WITH 1.0 WAS PADDED
SUBJECT Oslo24 WITH 1.0 WAS PADDED
SUBJECT Oslo24 WITH 2.0 WAS PADDED
SUBJECT Oslo25 WITH 11.0 WAS PADDED
SUBJECT Oslo26 WI

In [51]:
import numpy as np
from matplotlib import pyplot as plt

def clamp(num, mn, mx):
    """
    Clamps a number between two values.
    :param num: The number to clamp.
    :param mn: The minimum value.
    :param mx: The maximum value.
    :return: The clamped number.
    """
    return max(min(num, mx), mn)


seg_dir = '/data/projects/TMOR/data/Deeplearning/'
#indices = []
for subj_id in os.listdir(seg_dir):
    #extrasct the correct id from the subjects
    # Define paths to the segmentation and regular t1 postprocessed files for this subject
    seg_glob = os.path.join(seg_dir, f'{subj_id}', f'seg.nii.gz')
    reg_glob = os.path.join(seg_dir, f'{subj_id}', f't1_gd.nii.gz')
    seg_paths = glob.glob(seg_glob)
    reg_paths = glob.glob(reg_glob)
    # Loop through all matching segmentation and registration files
    for seg, reg in zip(seg_paths, reg_paths):
        if os.path.exists(seg) and os.path.exists(reg):
            mask = nib.load(seg)
            vol_mask = mask.get_fdata()
            for label in np.unique(vol_mask):
                if label != 0:
                    label_indices = np.argwhere(vol_mask == label)
                    if len(label_indices) > 125:
                        indices = label_indices
                        X_min, X_max = np.min(indices[:, 0]), np.max(indices[:, 0])
                        Y_min, Y_max = np.min(indices[:, 1]), np.max(indices[:, 1])
                        Z_min, Z_max = np.min(indices[:, 2]), np.max(indices[:, 2])

                        mean_x = (X_min + X_max) // 2
                        mean_y = (Y_min + Y_max) // 2
                        mean_z = (Z_min + Z_max) // 2
                        ###############################
                        d_x = D_x_max
                        d_y = D_y_max
                        d_z = D_z_max
                        ##################################
                        x_start = clamp(mean_x - d_x // 2, 0, vol_mask.shape[0])
                        x_end = clamp(mean_x + d_x // 2, 0, vol_mask.shape[0])

                        y_start = clamp(mean_y - d_y // 2, 0, vol_mask.shape[1])
                        y_end = clamp(mean_y + d_y // 2, 0, vol_mask.shape[1])

                        z_start = clamp(mean_z - d_z // 2, 0, vol_mask.shape[2])
                        z_end = clamp(mean_z + d_z // 2, 0, vol_mask.shape[2])

                        tumor_cutout = vol_mask[x_start:x_end, y_start:y_end, z_start:z_end]
                        ###################################
                        tensor = np.zeros([d_x, d_y, d_z])
                        # Define the position of the bounding box in the tensor
                        tx_start = abs(mean_x - d_x // 2) if mean_x - d_x // 2 < 0 else 0
                        tx_end = tumor_cutout.shape[0] + tx_start

                        ty_start = abs(mean_y - d_y // 2) if mean_y - d_y // 2 < 0 else 0
                        ty_end = tumor_cutout.shape[1] + ty_start

                        tz_start = abs(mean_z - d_z // 2) if mean_z - d_z // 2 < 0 else 0
                        tz_end = tumor_cutout.shape[2] + tz_start

                        # Place the bounding box in the tensor
                        tensor[tx_start:tx_end, ty_start:ty_end, tz_start:tz_end] = tumor_cutout
                        output_dir = os.path.join(seg_dir, 'B', subj_id)
                        os.makedirs(output_dir, exist_ok=True)
                         # Save the seg_partitioned into a file
                        output_path = os.path.join(output_dir, f'{label}seg_seg_.nii.gz')
                        nib.save(nib.Nifti1Image(tumor_cutout, mask.affine), output_path)


(134, 169, 141)
(152, 186, 143)
(145, 176, 140)
(133, 152, 126)
(138, 164, 137)
(132, 167, 137)
(130, 169, 139)
(143, 164, 142)
(142, 177, 141)
(157, 184, 148)
(138, 167, 138)
(126, 166, 120)
(138, 182, 141)
(142, 176, 135)
(143, 170, 141)
(138, 166, 125)
(139, 170, 132)
(142, 170, 136)
(138, 174, 132)
(130, 156, 132)
(140, 163, 137)
(149, 177, 139)
(132, 151, 129)
(132, 162, 132)
(135, 162, 127)
(140, 160, 135)
(140, 171, 125)
(140, 163, 141)
(130, 169, 133)
(143, 165, 130)
(142, 154, 135)
(139, 169, 140)
(148, 168, 143)
(140, 159, 137)
(134, 167, 134)
(138, 169, 132)
(139, 166, 135)
(134, 160, 129)
(143, 179, 143)
(144, 185, 140)
(130, 176, 136)
(135, 165, 128)
(137, 175, 129)
(147, 169, 131)
(139, 168, 135)
(138, 172, 132)
(132, 163, 131)
(147, 171, 140)
(145, 177, 147)
(146, 173, 143)
(142, 170, 135)
(141, 180, 140)
(124, 162, 129)
(150, 180, 148)
(148, 175, 143)
(130, 182, 137)
(143, 179, 136)
(138, 160, 143)
(131, 159, 133)
(133, 157, 135)
(130, 166, 128)
(130, 161, 126)
(135, 15

In [427]:
output_dir = os.path.join(seg_dir, 'A', subj_id)
os.makedirs(output_dir, exist_ok=True)
 # Save the seg_partitioned into a file
output_path = os.path.join(output_dir, f'seg_seg_.nii.gz')
nib.save(nib.Nifti1Image(seg_partitioned, mask.affine), output_path)

ImageFileError: Cannot work out file type of "/data/projects/TMOR/data/Deeplearning/A/Oslo02/seg_seg_.f6.0nii.gz"

In [98]:
df2.loc['Oslo66']

label                     1.000000
shape_Elongation          0.860610
shape_Flatness            0.856119
shape_LeastAxisLength     7.661783
shape_MajorAxisLength     8.949440
                           ...    
Age                      78.000000
number_annotations        1.000000
F                         1.000000
M                         0.000000
labels                    0.000000
Name: Oslo66, Length: 113, dtype: float64

In [440]:
class BrainSegmentationDataset(Dataset):
    def __init__(self, img_dir, labels_file, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.img_labels = pd.read_excel(labels_file)
        self.transform = transform
        self.target_transform = target_transform

        self.X_train, self.X_test, self.y_train, self.y_test = self.split_data()

    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, idx):
        if self.transform is None:
            # Load the mask directly
            seg_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 'seg.nii.gz')
            img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 't1_gd.nii.gz')
            label = self.y_train[idx]
            data = nib.load(img_path).get_fdata()
            mask = nib.load(seg_path).get_fdata()
        else:
            # Apply the transformation to the mask
            seg_path = os.path.join(self.img_dinr, self.img_labels.iloc[idx, 0], 'seg.nii.gz')
            img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 't1_gd.nii.gz')
            label = self.y_train[idx]
            mask = nib.load(seg_path).get_fdata()
            data = nib.load(img_path).get_fdata()

        mask = torch.from_numpy(mask).unsqueeze(0)
        label = torch.tensor(label).long()

        return mask, data,label

    def split_data(self):
        X = self.img_labels.iloc[:, 0].values
        y = self.img_labels.iloc[:, 2].values

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)


        return X_train, X_test, y_train, y_test


img_dir = '/data/projects/TMOR/data/Deeplearning/'
labels_file = 'labels_DL.xlsx'
dataset = BrainSegmentationDataset(img_dir, labels_file)

# Access the X_train list
print(dataset.X_train.size)

350


In [444]:
img_dir = '/data/projects/TMOR/data/Deeplearning/'
labels_file = 'labels_DL.xlsx'
transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])

dataset = BrainSegmentationDataset(img_dir,labels_file, transform=None)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
X_train

In [348]:
X_train, X_test, y_train, y_test = dataset.X_train, dataset.X_test, dataset.y_train, dataset.y_test

X_train[6].shape

torch.Size([1, 135, 159, 132])

In [403]:
# Assuming you have a DataFrame called "data"
# Features are stored in columns X1, X2, X3, ...
# The target variable is stored in column 'y'

X = new_df.drop(['Labels', 'Tumor'],axis=1)  # Features
y = new_df['Labels']  # Target variable
# Random split into 80% train and 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

img_dir = '/data/projects/TMOR/data/Deeplearning/'
labels_file = pd.read_excel('labels_DL.xlsx')

def process_images(img_dir, labels_file):
    image_tensors = []
    pene = list(labels_file.iloc[:, 0])
    for image_name in pene:
        seg_path = os.path.join(img_dir, image_name, 'seg.nii.gz')
        print(seg_path)
        img_path = os.path.join(img_dir, image_name, 't1_gd.nii.gz')
        mask = nib.load(seg_path).get_fdata()
        data = nib.load(img_path).get_fdata()

    data_tensor = torch.from_numpy(data).unsqueeze(0)
    mask_tensor = torch.from_numpy(mask).unsqueeze(0)

    image_tensors.append((data_tensor, mask_tensor))

    return image_tensors

processed_images = process_images(img_dir, labels_file)

/data/projects/TMOR/data/Deeplearning/Oslo01/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo01/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo02/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo02/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo03/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo03/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo04/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo06/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo07/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo08/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo09/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo09/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo10/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo11/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo12/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo13/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo14/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo17/seg

KeyboardInterrupt: 

In [431]:
pene = list(X_train.iloc[:, 0])

for i in pene:
    print(i)

Stan_265
Oslo62
Stan_197
Stan_058
Stan_319
Oslo47
Stan_026
Stan_165
Oslo31
Stan_260
Stan_049
Stan_312
Oslo41
Stan_045
Oslo26
Stan_251
Stan_285
Oslo13
Stan_051
Stan_028
Stan_183
Oslo32
Stan_064
Stan_203
Oslo46
Oslo06
Oslo37
Stan_009
Stan_315
Stan_024
Oslo20
Stan_184
Oslo12
Stan_126
Stan_120
Oslo01
Stan_144
Stan_019
Oslo49
Stan_054
Oslo33
Oslo45
Stan_291
Oslo30
Oslo28
Stan_134
Oslo44
Stan_059
Stan_102
Stan_318
Stan_246
Oslo52
Stan_148
Stan_010
Stan_038
Oslo38
Stan_121
Stan_136
Stan_171
Stan_014
Stan_087
Stan_098
Oslo60
Oslo07
Stan_132
Stan_170
Oslo54
Oslo40
Stan_052
Stan_047
Oslo39
Oslo51
Oslo09
Oslo48
Stan_033
Stan_307
Stan_072
Stan_055
Stan_289
Stan_290
Stan_068
Oslo10
Stan_142
Oslo17
Stan_005
Stan_257
Oslo04
Oslo21
Oslo43
Stan_037
Stan_266
Oslo08
Stan_244
Stan_101
Oslo03
Stan_173
Stan_016
Oslo61
Stan_123
Oslo55
Stan_323
Oslo66
Oslo53
Stan_066
Oslo25
Oslo65
Stan_316
Stan_227
Oslo42
Stan_311
Oslo02
Oslo59
Stan_230
Stan_111
Stan_100
Stan_149
Stan_065
Stan_039
Stan_176
Stan_338
Oslo24
Sta

In [419]:
X_train.iloc[4, :]

Subjid    Stan_319
Name: 2369, dtype: object